In [145]:
import pandas as pd
import json
import numpy as np
from itertools import product

In [378]:
pres = pd.read_csv(open('1976-2016-president.csv'))

In [379]:
def clean_pres(in_df):
    df = in_df.copy()
    df = df[~df['candidate'].isna()]
    df = df[df['writein'] == False]
    df = df.drop(columns=['notes', 'version', 'writein', 'state', 'state_cen', 'state_ic'])
    df = df[df['party'].isin(['democrat', 'republican', 'democratic-npl', 'democratic-farmer-labor'])]
    df = df.replace('democratic-farmer-labor', 'D')
    df = df.replace('democratic-npl', 'D')
    df = df.replace('democrat', 'D')
    df = df.replace('republican', 'R')
    df['office'] = 'president'
    return df.reset_index(drop=True)

In [380]:
ppres = clean_pres(pres)

In [381]:
sen = pd.read_csv(open('1976-2018-senate.csv'))

In [382]:
def clean_sen(in_df):
    df = in_df.copy()
    df = df[~df['candidate'].isna()]
    df = df[df['writein'] == False]
    df = df[df['stage'] == 'gen']
    df = df[(df['candidatevotes']/df['totalvotes']) > 0.1]
    df = df.drop(columns=['version', 'writein', 'state', 'state_cen', 'state_ic', 'district', 'mode', 'stage', 'unofficial'])
    df = df[df['party'].isin(['democrat', 'republican', 'democratic-npl', 'democratic-farmer-labor'])]
    df = df.replace('democratic-farmer-labor', 'D')
    df = df.replace('democratic-npl', 'D')
    df = df.replace('democrat', 'D')
    df = df.replace('republican', 'R')
    df['office'] = 'senate'
    return df.reset_index(drop=True)

In [383]:
ssen = clean_sen(sen)

In [420]:
def calc_sen_d_prob(in_df):
    df = in_df.copy()
    races = np.unique(in_df[['year', 'state_po', 'special']].values.astype(str), axis=0)
    invalid_indices = []
    for year, state, special in races:
        rows = df[(df['year'] == int(year)) & (df['state_po'] == state) & (df['special'] == eval(special))]
        d_candidate = rows[rows['party'] == 'D']
        r_candidate = rows[rows['party'] == 'R']
        if len(d_candidate) != 1 or len(r_candidate) != 1:
            invalid_indices.extend(d_candidate.index)
            invalid_indices.extend(r_candidate.index)
            continue
    df = df.drop(index=invalid_indices)
    df = df[df['party'] == 'D']
    df['d_prob'] = df['candidatevotes'] / df['totalvotes']
#         d_count = d_candidate['candidatevotes'].item()
#         r_count = r_candidate['candidatevotes'].item()
#         total_d_r =  d_count + r_count
#         df.loc[d_candidate.index]['ratio'] = d_count / total_d_r
#         df.loc[r_candidate.index]['ration'] = r_count / total_d_r  
    return df[['year', 'state_po', 'state_fips', 'office', 'd_prob']].reset_index(drop=True)

In [421]:
d_prob_senate = calc_sen_d_prob(ssen)

In [422]:
d_prob_senate.to_csv(open('1976-2018-senate-d-prob.csv', 'w+'))

In [417]:
def calc_pres_d_prob(in_df):
    df = in_df.copy()
    races = np.unique(in_df[['year', 'state_po']].values.astype(str), axis=0)
    invalid_indices = []
    for year, state in races:
        rows = df[(df['year'] == int(year)) & (df['state_po'] == state)]
        assert(len(rows) == 2)
    df = df[df['party'] == 'D']
    df['d_prob'] = df['candidatevotes'] / df['totalvotes']
#         d_count = d_candidate['candidatevotes'].item()
#         r_count = r_candidate['candidatevotes'].item()
#         total_d_r =  d_count + r_count
#         df.loc[d_candidate.index]['ratio'] = d_count / total_d_r
#         df.loc[r_candidate.index]['ration'] = r_count / total_d_r  
    return df[['year', 'state_po', 'state_fips', 'office', 'd_prob']].reset_index(drop=True)

In [418]:
d_prob_pres = calc_pres_d_prob(ppres)

In [419]:
d_prob_pres.to_csv(open('1976-2016-president-d-prob.csv', 'w+'))

In [394]:
house = pd.read_csv(open('1976-2018-house2.csv'))

In [395]:
house

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Alabama,AL,1,63,41,US House,1,gen,False,False,Bill Davenport,democrat,False,total,58906,157170,False,20200424
1,1976,Alabama,AL,1,63,41,US House,1,gen,False,False,Jack Edwards,republican,False,total,98257,157170,False,20200424
2,1976,Alabama,AL,1,63,41,US House,1,gen,False,False,NaN,NaN,True,total,7,157170,False,20200424
3,1976,Alabama,AL,1,63,41,US House,2,gen,False,False,J. Carole Keahey,democrat,False,total,66288,156362,False,20200424
4,1976,Alabama,AL,1,63,41,US House,2,gen,False,False,NaN,NaN,True,total,5,156362,False,20200424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29631,2018,Maryland,MD,24,52,52,US House,7,gen,NaN,False,NaN,NaN,True,total,272,264710,False,20200424
29632,2018,Maryland,MD,24,52,52,US House,8,gen,NaN,False,Jamie Raskin,democrat,False,total,217679,319330,False,20200424
29633,2018,Maryland,MD,24,52,52,US House,8,gen,NaN,False,Jasen Wunder,libertarian,False,total,4853,319330,False,20200424
29634,2018,Maryland,MD,24,52,52,US House,8,gen,NaN,False,John Walsh,republican,False,total,96525,319330,False,20200424


In [406]:
def clean_house(in_df):
    df = in_df.copy()
    df = df[~df['candidate'].isna()]
    df = df[df['writein'] == False]
    df = df[df['stage'] == 'gen']
    df = df[(df['candidatevotes']/df['totalvotes']) > 0.1]
    df = df.drop(columns=['version', 'writein', 'state', 'state_cen', 'state_ic', 'mode', 'stage', 'unofficial'])
    df = df[df['party'].isin(['democrat', 'republican', 'democratic-npl', 'democratic-farmer-labor'])]
    df = df.replace('democratic-farmer-labor', 'D')
    df = df.replace('democratic-npl', 'D')
    df = df.replace('democrat', 'D')
    df = df.replace('republican', 'R')
    df['office'] = 'house'
    return df.reset_index(drop=True)

In [407]:
clean_house(house)

,year,state_po,state_fips,office,district,runoff,special,candidate,party,candidatevotes,totalvotes
0,1976,AL,1,house,1,False,False,Bill Davenport,D,58906,157170
1,1976,AL,1,house,1,False,False,Jack Edwards,R,98257,157170
2,1976,AL,1,house,2,False,False,J. Carole Keahey,D,66288,156362
3,1976,AL,1,house,2,False,False,"William L. """"Bill"""" Dickinson",R,90069,156362
4,1976,AL,1,house,3,False,False,Bill Nichols,D,106935,108048
...,...,...,...,...,...,...,...,...,...,...,...
17739,2018,MD,24,house,6,NaN,False,David Trone,D,163346,276974
17740,2018,MD,24,house,7,NaN,False,Elijah Cummings,D,202345,264710
17741,2018,MD,24,house,7,NaN,False,Richmond Davis,R,56266,264710
17742,2018,MD,24,house,8,NaN,False,Jamie Raskin,D,217679,319330


In [410]:
hhouse = clean_house(house)

In [415]:
def calc_house_d_prob(in_df):
    df = in_df.copy()
    races = np.unique(in_df[['year', 'state_po', 'district', 'special']].values.astype(str), axis=0)
    invalid_indices = []
    for year, state, district, special in races:
        rows = df[(df['year'] == int(year)) & (df['state_po'] == state) & (df['district'] == district) & (df['special'] == eval(special))]
        d_candidate = rows[rows['party'] == 'D']
        r_candidate = rows[rows['party'] == 'R']
        if len(d_candidate) != 1 or len(r_candidate) != 1:
            invalid_indices.extend(d_candidate.index)
            invalid_indices.extend(r_candidate.index)
            continue
    df = df.drop(index=invalid_indices)
    df = df[df['party'] == 'D']
    df['d_prob'] = df['candidatevotes'] / df['totalvotes']
    return df[['year', 'state_po', 'state_fips', 'district', 'office', 'd_prob']].reset_index(drop=True)

In [423]:
d_prob_house = calc_house_d_prob(hhouse)

[]


In [424]:
d_prob_house.to_csv(open('1976-2018-house-d-prob.csv', 'w+'))